# Important links for this project

1- https://towardsdatascience.com/leveraging-on-nlp-to-gain-insights-in-social-media-news-broadcasting-ca89752ef638

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')

import re
import string
import numpy as np
import pandas as pd
from pprint import pprint

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize

# import gensim
# import pyLDAvis
# import pyLDAvis.gensim_models
# import gensim.corpora as corpora
# from gensim.models import CoherenceModel


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zainn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zainn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
AU = pd.read_csv('data_AU.csv')
UK = pd.read_csv('data_UK.csv')
NZ = pd.read_csv('data_NZ.csv')
IE = pd.read_csv('data_IE.csv')
DE = pd.read_csv('data_DE.csv')
FR = pd.read_csv('data_FR.csv')
CA = pd.read_csv('data_CA.csv')

In [8]:
AU

,title,text,published_date
0,Reliance to raise $5 billion to retire existin...,Mukesh Ambani-led Reliance Industries Ltd (RIL...,2022-01-01 21:12:18+05:30
1,Vinod Kannan takes over as Vistara CEO,Vistara on Saturday said that Vinod Kannan too...,2022-01-02 03:33:24+05:30
2,"Zomato, Swiggy to collect 5% GST beginning Jan...",Food aggregators like Swiggy and Zomato will h...,2022-01-01 16:21:50+05:30
3,SpaceX creating the noise of progress in McGregor,McGregor City Council Member Steven Dutschmann...,None
4,Tata Motors' passenger vehicles business hive-...,The hiving off and transfer of Tata Motors' en...,2022-01-01 18:50:47+05:30
...,...,...,...
34959,WHO vows to support DOH against monkeypox,The World Health Organization assured that it ...,2022-07-30 18:49:34+08:00
34960,Time spent playing video games unlikely to aff...,Despite widespread concerns about the potentia...,2022-07-30 22:26:00-04:00
34961,Danny Garcia reveals mental health struggles a...,Danny Garcia won his first match at super welt...,None
34962,N.L. virologist doesn't think monkeypox will b...,"Rod Russell, a professor who studies viruses a...",None


In [18]:
AU.columns

Index(['title', 'text', 'published_date'], dtype='object')

In [29]:
df = pd.DataFrame(columns=AU.columns)
df = df.append([AU, UK, NZ, IE, DE, FR,CA], ignore_index=True)
df.drop('published_date', axis=1, inplace=True)
df = df.drop_duplicates()
df.reset_index(inplace=True)

In [30]:
df

,index,title,text
0,0,Reliance to raise $5 billion to retire existin...,Mukesh Ambani-led Reliance Industries Ltd (RIL...
1,1,Vinod Kannan takes over as Vistara CEO,Vistara on Saturday said that Vinod Kannan too...
2,2,"Zomato, Swiggy to collect 5% GST beginning Jan...",Food aggregators like Swiggy and Zomato will h...
3,3,SpaceX creating the noise of progress in McGregor,McGregor City Council Member Steven Dutschmann...
4,4,Tata Motors' passenger vehicles business hive-...,The hiving off and transfer of Tata Motors' en...
...,...,...,...
79133,226584,Courage 2 Go Further Campaign,Health\n\nCourage 2 Go Further Campaign\n\nCoa...
79134,226585,City records 97th COVID-related death,Health Unit announces 30 new confirmed COVID c...
79135,226590,Why We Don't Need A New Abortion Law In Canada,Drafted in partnership with the National Assoc...
79136,226598,Just in time for summer: 2 Vancouver beaches c...,With hot weather on the way this weekend for t...


In [2]:
words_to_remove = ['say', 'news', 'year', 'make', 'time', 'also', 'would', 'take', 'go', 'like', 'first', 'come', 'told', 'last', 'know', 'right', 'call', 'could', 'week', 'include', 'well', 'back', 'want', 'need', 'even', 'many', 'court', 'school', 'month', 'world', 'high', 'add', 
'article', 'give', 'look', 'listen', 'since', 'click', 'medium', 'still', 'thing', 'three', 'continue', 'use', 'change', 'much', 'around', 'write', 'long', 'number', 'next', 'face', 'another', 'york', 'move', 'really', 'see', 'found', 'ask', 'talk', 'become','left', 
'every', 'good', 'question', 'tuesday', 'provide', 'second', 'begin', 'record', 'day', 'read', 'name', 'try', 'late', 'monday', 'thursday', 'never', 'friday', 'happen', 'image', 'january', 'wednesday', 'control', 'later', 'four', 'something']
   

In [31]:
en_stopwords = stopwords.words('english')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_word(word):
    return WordNetLemmatizer().lemmatize(word, get_wordnet_pos(word))

def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    return tokenizer.tokenize(text)

def remove_whitespace(text):
    return " ".join(text.split())

def remove_tag(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', str(text))

def remove_urls(text):
    html_pattern = re.compile('https?://\S+|www\.\S+')
    return html_pattern.sub(r'', text)

def miscellaneous(word):
    if word not in en_stopwords and len(word)>3 and not word.isdigit():
        return lemmatize_word(word)

def pre_process(text, i):
    print('On row: '+ str(i))
    
    text = remove_tag(text)
    text = remove_urls(text)
    text = str(text).lower()
    text = remove_whitespace(text)
    text = remove_punct(text)
    text = list(map(miscellaneous, text))
    text = list(filter(None.__ne__, text))
    return ' '.join(text)

def check_word(word):
    if word not in words_to_remove:
        return word

def remove_words(text, i):
    print('On row: '+ str(i))
    text = str(text)
    text = list(map(check_word, text.split()))
    text = list(filter(None.__ne__, text))
    return ' '.join(text)

In [32]:
processed_df = pd.DataFrame(columns=['text'])
processed_df['text'] = list(map(pre_process, df['text'], df.index))

On row: 0
On row: 1
On row: 2
On row: 3
On row: 4
On row: 5
On row: 6
On row: 7
On row: 8
On row: 9
On row: 10
On row: 11
On row: 12
On row: 13
On row: 14
On row: 15
On row: 16
On row: 17
On row: 18
On row: 19
On row: 20
On row: 21
On row: 22
On row: 23
On row: 24
On row: 25
On row: 26
On row: 27
On row: 28
On row: 29
On row: 30
On row: 31
On row: 32
On row: 33
On row: 34
On row: 35
On row: 36
On row: 37
On row: 38
On row: 39
On row: 40
On row: 41
On row: 42
On row: 43
On row: 44
On row: 45
On row: 46
On row: 47
On row: 48
On row: 49
On row: 50
On row: 51
On row: 52
On row: 53
On row: 54
On row: 55
On row: 56
On row: 57
On row: 58
On row: 59
On row: 60
On row: 61
On row: 62
On row: 63
On row: 64
On row: 65
On row: 66
On row: 67
On row: 68
On row: 69
On row: 70
On row: 71
On row: 72
On row: 73
On row: 74
On row: 75
On row: 76
On row: 77
On row: 78
On row: 79
On row: 80
On row: 81
On row: 82
On row: 83
On row: 84
On row: 85
On row: 86
On row: 87
On row: 88
On row: 89
On row: 90
On row: 9

KeyboardInterrupt: 

In [4]:
processed_df = pd.read_csv('procesed_data.csv')
clean_df = pd.DataFrame(columns=processed_df.columns)

In [5]:
processed_df

,title,text,published_date
0,virginia teacher union mocked grammatical erro...,listen news article virginia teacher union fac...,NaN
1,desantis tout freest state florida appearance ...,listen news article florida desantis wife case...,NaN
2,multiple medium outlet paint dark picture head...,listen news article already rough start base c...,NaN
3,midterm wave overcome blue wall,listen news article chair congressional reelec...,NaN
4,omicron variant impact lung differently study,listen news article recent study suggest omicr...,NaN
...,...,...,...
16221,ivana hrynkiw shatara alabama reporter bar pri...,ivana hrynkiw shatara video news producer clai...,NaN
16222,noam chomsky censorship bad soviet russia,noam chomsky russel brand discus american cens...,NaN
16223,katie halper trump broke liberal brain,comedian host popular progressive podcasts off...,NaN
16224,kamala harris supporter come defense accuse mo...,kamala harris supporter come defense critic ac...,NaN


In [7]:
clean_df['title'] = list(map(remove_words, processed_df.title, processed_df.index))

On row: 0
On row: 1
On row: 2
On row: 3
On row: 4
On row: 5
On row: 6
On row: 7
On row: 8
On row: 9
On row: 10
On row: 11
On row: 12
On row: 13
On row: 14
On row: 15
On row: 16
On row: 17
On row: 18
On row: 19
On row: 20
On row: 21
On row: 22
On row: 23
On row: 24
On row: 25
On row: 26
On row: 27
On row: 28
On row: 29
On row: 30
On row: 31
On row: 32
On row: 33
On row: 34
On row: 35
On row: 36
On row: 37
On row: 38
On row: 39
On row: 40
On row: 41
On row: 42
On row: 43
On row: 44
On row: 45
On row: 46
On row: 47
On row: 48
On row: 49
On row: 50
On row: 51
On row: 52
On row: 53
On row: 54
On row: 55
On row: 56
On row: 57
On row: 58
On row: 59
On row: 60
On row: 61
On row: 62
On row: 63
On row: 64
On row: 65
On row: 66
On row: 67
On row: 68
On row: 69
On row: 70
On row: 71
On row: 72
On row: 73
On row: 74
On row: 75
On row: 76
On row: 77
On row: 78
On row: 79
On row: 80
On row: 81
On row: 82
On row: 83
On row: 84
On row: 85
On row: 86
On row: 87
On row: 88
On row: 89
On row: 90
On row: 9

In [8]:
clean_df['text'] = list(map(remove_words, processed_df.text, processed_df.index))

On row: 0
On row: 1
On row: 2
On row: 3
On row: 4
On row: 5
On row: 6
On row: 7
On row: 8
On row: 9
On row: 10
On row: 11
On row: 12
On row: 13
On row: 14
On row: 15
On row: 16
On row: 17
On row: 18
On row: 19
On row: 20
On row: 21
On row: 22
On row: 23
On row: 24
On row: 25
On row: 26
On row: 27
On row: 28
On row: 29
On row: 30
On row: 31
On row: 32
On row: 33
On row: 34
On row: 35
On row: 36
On row: 37
On row: 38
On row: 39
On row: 40
On row: 41
On row: 42
On row: 43
On row: 44
On row: 45
On row: 46
On row: 47
On row: 48
On row: 49
On row: 50
On row: 51
On row: 52
On row: 53
On row: 54
On row: 55
On row: 56
On row: 57
On row: 58
On row: 59
On row: 60
On row: 61
On row: 62
On row: 63
On row: 64
On row: 65
On row: 66
On row: 67
On row: 68
On row: 69
On row: 70
On row: 71
On row: 72
On row: 73
On row: 74
On row: 75
On row: 76
On row: 77
On row: 78
On row: 79
On row: 80
On row: 81
On row: 82
On row: 83
On row: 84
On row: 85
On row: 86
On row: 87
On row: 88
On row: 89
On row: 90
On row: 9

In [9]:
clean_df.to_csv('procesed_data_words_remove.csv', index=False)